In [ ]:
from pathlib import Path
import shutil
from tqdm import tqdm
from ultralytics import YOLO
import cv2
from pathlib import Path

In [ ]:
model_name = "yolo11n.pt"
image_directory = Path("images")
out_directory = Path("results")

exts = {".jpg", ".jpeg", ".png", ".bmp"}
image_paths = [p for p in image_directory.iterdir() if p.suffix.lower() in exts]

In [ ]:
model = YOLO(model_name)
# img = cv2.imread(image_directory) # 1 afbeelding

In [ ]:
SHOW       = True           # zet True als je elke image wil tonen
for img_path in image_paths:
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Skip (niet leesbaar): {img_path}")
        continue

    results = model(img)  # detectie
    r = results[0]

    # Console output
    names = r.names
    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(float, box.xyxy[0].tolist())
        print(f"{img_path.name}: {names[cls]} {conf:.2f} [{x1:.0f},{y1:.0f},{x2:.0f},{y2:.0f}]")

    # Annoteren + opslaan
    annotated = r.plot()
    out_path = out_directory / f"{img_path.stem}_det{img_path.suffix}"
    cv2.imwrite(str(out_path), annotated)

    if SHOW:
        cv2.imshow("YOLOv11", annotated)
        cv2.waitKey(0)

if SHOW:
    cv2.destroyAllWindows()

print("Klaar Resultaten staan in:", out_directory)

In [ ]:
# Paden moeten wijzigen want dit was lokaal
SOURCE_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\CARLA.v20-carla_v20.yolov11")
TARGET_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla_acc8")

SRC_CLASSES = [
    'bike', 'motobike', 'person',
    'traffic_light_green', 'traffic_light_orange', 'traffic_light_red',
    'traffic_sign_30', 'traffic_sign_60', 'traffic_sign_90',
    'vehicle'
]

MAP_TO_TARGET = {
    'bike': 5, 'motobike': 4, 'person': 3,
    'traffic_light_green': 6, 'traffic_light_orange': 6, 'traffic_light_red': 6,
    'traffic_sign_30': 7, 'traffic_sign_60': 7, 'traffic_sign_90': 7,
    'vehicle': 0
}

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

In [ ]:
def find_image(src_images_dir: Path, stem: str):
    for ext in IMG_EXTS:
        p = src_images_dir / f"{stem}{ext}"
        if p.exists():
            return p
    return None

In [ ]:
def remap_split(split: str, stats):

    # bronmappen
    src_img_dir = SOURCE_DIR / split / "images"
    src_lbl_dir = SOURCE_DIR / split / "labels"

    # doelmappen (jij hebt deze al aangemaakt)
    dst_img_dir = TARGET_DIR / "images" / ("train" if split == "train" else "val")
    dst_lbl_dir = TARGET_DIR / "labels" / ("train" if split == "train" else "val")

    lbl_files = list(src_lbl_dir.glob("*.txt"))

    for lbl_path in tqdm(lbl_files, desc=f"Processing {split}"):
        stem = lbl_path.stem
        img_path = find_image(src_img_dir, stem)

        if img_path is None:
            stats["missing_image"] += 1
            continue

        lines_out = []
        raw = lbl_path.read_text().strip().splitlines()

        for line in raw:
            parts = line.split()
            if len(parts) != 5:
                continue
            src_cls_idx = int(parts[0])
            src_name = SRC_CLASSES[src_cls_idx]

            if src_name not in MAP_TO_TARGET:
                continue

            tgt_cls = MAP_TO_TARGET[src_name]
            parts[0] = str(tgt_cls)
            lines_out.append(" ".join(parts))

            stats["per_class"][tgt_cls] = stats["per_class"].get(tgt_cls, 0) + 1

        # Image kopiëren
        shutil.copy2(img_path, dst_img_dir / img_path.name)

        # Labelbestand schrijven (mag leeg zijn)
        (dst_lbl_dir / lbl_path.name).write_text("\n".join(lines_out) + ("\n" if lines_out else ""))

        stats["images"] += 1
        if lines_out:
            stats["labeled_images"] += 1
        else:
            stats["empty_labels"] += 1


In [ ]:
stats = {"images":0, "labeled_images":0, "empty_labels":0, "missing_image":0, "per_class":{}}

splits = []
if (SOURCE_DIR / "train").exists(): splits.append("train")
if (SOURCE_DIR / "val").exists():   splits.append("val")
if (SOURCE_DIR / "valid").exists(): splits.append("valid")  # Roboflow  'valid'

for split in splits:
    remap_split(split, stats)

print("\n=== Samenvatting ===")
print(stats)

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)
model.train(
    data=r"C:/Users/Khatc/Desktop/DistributedAI/Project/Code_Train/dataset/Carla_acc8/data_acc8.yaml",
    epochs=2,        # korte rooktest
    imgsz=640,
    batch=4,
    device="cpu"
)

